In [2]:
import sys
sys.path.append('/cellar/users/hsher/KIR_development')

from normalization.coverage import four_mer, four_mer_coverage
import pandas as pd
four_mers = four_mer()

In [6]:
# select only the qualified k-mers
from normalization.kmer_selection import select_qualified_kmer
kmer_df = pd.read_csv('~/Data/kmer_df', header = 0)
kmer_df = kmer_df.loc[kmer_df['length'] == 20]
selected_kmer = select_qualified_kmer(kmer_df['kmer'])
kmer_df = kmer_df.loc[kmer_df['kmer'].isin(selected_kmer)]

gene_kmer_count = kmer_df.groupby(by = 'gene').count()['kmer']

# Normalize by 4-mer estimated coverage

In [23]:
# parallel by 
def normalize(series_index):
    return(q_kmer.loc[series_index]/four_cov[series_index])
def normalize_4mer(df):
    # make sure no dups
    df.drop_duplicates(inplace = True) # weird dbGap kmer has duplicates
    
    # retrieve four mer coverages
    global four_cov
    four_cov = four_mer_coverage(df, four_mers)

    print('done retrieving normalization')
    # retrieve qualifies k-mer
    global q_kmer
    q_kmer = df[selected_kmer]
    
    
    
    # normalize by four-mer estimated coverage or KIR reads
    from multiprocessing import Pool, Manager
    
    
    with Pool(8) as p:
        normalized = p.map(normalize, q_kmer.index)
    
    return(pd.concat(normalized, axis = 1).transpose())    
        


In [24]:
d = pd.read_csv('~/Data/kmers/dbGap_kmer.csv', header = 0, index_col= 0)
d_out = normalize_4mer(d)

done retrieving normalization


In [27]:
d_out.to_csv('~/Data/normalized_kmer/nor_dbGap.csv')

In [26]:
d_out.shape

(1635, 2868)

In [28]:
d = pd.read_csv('~/Data/kmers/KhoeSan_kmers.csv', header = 0, index_col= 0)
d_out = normalize_4mer(d)
d_out.to_csv('~/Data/normalized_kmer/nor_KhoeSan.csv')

done retrieving normalization


# Normalize by number of qualified k-mer


In [30]:
from multiprocessing import Pool
def normalize(col):
    return(df[col]/denominator)

def normalize_col(gene):
    cols = kmer_df.loc[kmer_df['gene'] == gene, 'kmer']
    global denominator
    denominator = gene_kmer_count[gene]
    with Pool(16) as p:
        series = p.map(normalize, cols)
    return(pd.concat(series, axis = 1))

def normalize_by_kmer(df, output_name):
    output_df = pd.DataFrame(index = df.index)
    # normalize by count of k-mer
    for gene in gene_kmer_count.index:
        concat_df = normalize_col(gene)
        output_df = output_df.join(concat_df)
    output_df.to_csv(output_name)

In [32]:
df = pd.read_csv('~/Data/normalized_kmer/nor_KhoeSan.csv', header = 0, index_col = 0)
normalize_by_kmer(df, '~/Data/normalized_kmer/nornor_KhoeSan.csv')


In [33]:
df = pd.read_csv('~/Data/normalized_kmer/nor_dbGap.csv', header = 0, index_col = 0)
normalize_by_kmer(df, '~/Data/normalized_kmer/nornor_dbGap.csv')
